<a href="https://colab.research.google.com/github/LostRuins/koboldcpp/blob/concedo/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Welcome to the Official KoboldCpp Colab Notebook
It's really easy to get started. Just press the two **Play** buttons below, and then connect to the **Cloudflare URL** shown at the end. 
You can also change the model URL to use any GGUF model from Huggingface.

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldCpp below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title <b>v-- Enter your model below and then click this to start Koboldcpp</b>

Model = "https://huggingface.co/KoboldAI/LLaMA2-13B-Tiefighter-GGUF/resolve/main/LLaMA2-13B-Tiefighter.Q4_K_M.gguf" #@param [""]{allow-input: true}
Layers = 43 #@param [43]{allow-input: true}
ContextSize = 4096 #@param [4096] {allow-input: true}

%cd /content
!git clone https://github.com/LostRuins/koboldcpp
%cd /content/koboldcpp
kvers = !(cat koboldcpp.py | grep 'KcppVersion = ' | cut -d '"' -f2)
kvers = kvers[0]
!echo Finding prebuilt binary for {kvers}
!wget -c https://huggingface.co/concedo/koboldcpp/resolve/main/prebuilt_binaries/{kvers}.so
!test -f {kvers}.so && mv {kvers}.so koboldcpp_cublas.so || echo Prebuilt Binary Does Not Exist
!test -f koboldcpp_cublas.so && echo Prebuilt Binary Exists || make koboldcpp_cublas LLAMA_CUBLAS=1
!cp koboldcpp_cublas.so koboldcpp_cublas.dat
!wget $Model -O model.ggml
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:5001 &
!sleep 8
!cat nohup.out
!python koboldcpp.py model.ggml --usecublas 0 mmq --multiuser --gpulayers $Layers --contextsize $ContextSize --hordeconfig concedo 1 1 --onready "echo Connect to the link below && cat nohup.out | grep trycloudflare.com && rm nohup.out"
